In [47]:
class PDBLINE(dict):
    """
    Create a single PDB line and write its fields.
    """
    def __init__(self, input_line, **kwargs):
        super().__init__()
        self.input_line = input_line
        # keep track if the line is an ATOM or HETATM
        self.is_atom = True
        # should actually make field_templates private Standart pdb
        # definition, changed to make it zero based and compatible
        # with python indexing by subtracting one from the left index
        self.field_templates = {
            'type':        ([0, 6],     '“ATOM”',		                     'character'),
            'atomid':      ([6, 11],	'Atom serial number	right',	         'integer'),
            'atomname':    ([12, 16],	'Atom name	left',	                 'character'),
            'altlocid':    ([16, 17],   'Alternate location indicator',	     'character'),
            'resname':     ([17, 20],	'Residue name	right',	             'character'),
            'chainid':     ([21, 22],   'Chain identifier',		             'character'),
            'resid':       ([22, 26],	'Residue sequence number	right',	 'integer'),
            'code':        ([26, 27],   'Code for insertions of residues',	 'character'),
            'posx':        ([30, 38],	'X orthogonal A coordinate	right',  'real (8.3)'),
            'posy':        ([38, 46],	'Y orthogonal A coordinate	right',  'real (8.3)'),
            'posz':        ([46, 54],	'Z orthogonal A coordinate	right',  'real (8.3)'),
            'occupancy':   ([54, 60],	'Occupancy	right',	                 'real (6.2)'),
            'tempfact':    ([60, 66],	'Temperature factor	right',	         'real (6.2)'),
            'segid':       ([72, 76],	'Segment identifier	left',	         'character'),
            'element':     ([76, 78],	'Element symbol	right',	             'character'),
        }
        for key, item in kwargs.items():
            self.__setitem__(key, item)

    @staticmethod
    def from_line(pdb_line):
        """ Construct a PDBLINE obj from a full pdb line given as a str. """
        # if not a proper atom line, mark and return
        if pdb_line[ : 4].strip() not in ['ATOM', 'HETATM']:
            # since I do not fix numbering on TER lines jet, just erase it
            if pdb_line.startswith('TER'):
                pdb_line = 'TER\n'
            to_construct = PDBLINE(pdb_line)
            to_construct.is_atom = False
            return to_construct
        # pad or cut line to len 79
        if len(pdb_line) < 79:
            pdb_line += ' ' * (79 - len(pdb_line))
        pdb_line = pdb_line[  : 79]
        to_construct = PDBLINE(pdb_line)
        for key, field_descriptor in to_construct.field_templates.items():
            idx_start, idx_end = field_descriptor[0]
            to_construct[key] = pdb_line[idx_start : idx_end]
        return to_construct

    def get_str_with_len(self, content, length):
        """ Take the content and pad it with spaces from the left until
        if has length. Asserts that str(content) <= length.
        """
        out_str = str(content)
        assert len(out_str) <= length, f'content too long: {content}, {length}'
        while len(out_str) < length:
            out_str = ' ' + out_str
        return out_str

    def __setitem__(self, key, item):
        idxs = self.field_templates[key][0]
        len_field = idxs[1] - idxs[0]
        assert len(str(item)) <= len_field, (
            'The given item to be written into a PDBLINE does not have the right '
            'length. When cast to a string, it should have less than or equal '
            f'than {len_field}, but it has: {len(str(item))}.'
        )
        super().__setitem__(key, self.get_str_with_len(item, len_field))

    def sub_line(self, idx1, idx2, word, line):
        """ Given a pdb line string, substitute a single fields with word. """
        assert idx2 - idx1 == len(word), f'{idx1 - idx2} {len(word)}'
        for i, idx in enumerate(range(idx1, idx2)):
            line[idx] = word[i]

    def get_line(self, pad_with=' \n'):
        """ Write all fields into a line, empty fields end up as spaces.
        If the line is not of type ATOM or HETATOM, return unmodified line. """
        if not self.is_atom:
            return self.input_line
        line = 79 * [' ']
        for key, (colums, __, __) in self.field_templates.items():
            if key in super().keys():
                word = self.get_str_with_len(super().__getitem__(key), colums[1] - colums[0])
                self.sub_line(colums[0], colums[1], word, line)
        return ''.join(line) + pad_with

    def __repr__(self):
        return self.get_line().strip('\n')

In [48]:
def fix_atom_numbering(pdb_lines):
    """ Check that the atom numbering is increasing by one each line,
    given a list of pdb lines. Returns a bopy of pdb_lines.
    Arguments:
        pdblines (list of str): List containing the pdb lines to be changed.
    """
    output_lines = []
    for i, line in enumerate(pdb_lines):
        line_obj = PDBLINE.from_line(line)
        if line_obj.is_atom:
            line_obj['atomid'] = i + 1
        output_lines.append(line_obj.get_line())
    return output_lines

In [49]:
PDBLINE.from_line(lines[2]).get_line()

'ATOM     37  CA  ASN A   3       1.900  10.505   4.185  1.00  0.00           C  \n'

In [50]:
lines = [x for x in open('residue_templates/asn.pdb').readlines() if x.startswith('A')]
lines = open('residue_templates/asn.pdb').readlines()
lines

['MODEL        0\n',
 'ATOM     36  N   ASN A   3       0.435  10.506   4.227  1.00  0.00           N  \n',
 'ATOM     37  CA  ASN A   3       1.900  10.505   4.185  1.00  0.00           C  \n',
 'ATOM     38  C   ASN A   3       2.363  10.506   5.654  1.00  0.00           C  \n',
 'ATOM     39  O   ASN A   3       1.576  10.506   6.602  1.00  0.00           O  \n',
 'ATOM     40  CB  ASN A   3       2.428  11.737   3.449  1.00  0.00           C  \n',
 'ATOM     41  HB3 ASN A   3       1.906  11.863   2.493  1.00  0.00           H  \n',
 'ATOM     42  HB2 ASN A   3       2.258  12.643   4.043  1.00  0.00           H  \n',
 'ATOM     43  CG  ASN A   3       3.903  11.631   3.141  1.00  0.00           C  \n',
 'ATOM     44  OD1 ASN A   3       4.600  10.655   3.393  1.00  0.00           O  \n',
 'ATOM     45  ND2 ASN A   3       4.440  12.746   2.570  1.00  0.00           N  \n',
 'ATOM     46 HD21 ASN A   3       3.884  13.540   2.280  1.00  0.00           H  \n',
 'ATOM     47 HD22 ASN

In [51]:
# fix_atom_numbering(fix_residue_numbering(lines))
[x for x in fix_atom_numbering(lines)]

['MODEL        0\n',
 'ATOM      2  N   ASN A   3       0.435  10.506   4.227  1.00  0.00           N  \n',
 'ATOM      3  CA  ASN A   3       1.900  10.505   4.185  1.00  0.00           C  \n',
 'ATOM      4  C   ASN A   3       2.363  10.506   5.654  1.00  0.00           C  \n',
 'ATOM      5  O   ASN A   3       1.576  10.506   6.602  1.00  0.00           O  \n',
 'ATOM      6  CB  ASN A   3       2.428  11.737   3.449  1.00  0.00           C  \n',
 'ATOM      7  HB3 ASN A   3       1.906  11.863   2.493  1.00  0.00           H  \n',
 'ATOM      8  HB2 ASN A   3       2.258  12.643   4.043  1.00  0.00           H  \n',
 'ATOM      9  CG  ASN A   3       3.903  11.631   3.141  1.00  0.00           C  \n',
 'ATOM     10  OD1 ASN A   3       4.600  10.655   3.393  1.00  0.00           O  \n',
 'ATOM     11  ND2 ASN A   3       4.440  12.746   2.570  1.00  0.00           N  \n',
 'ATOM     12 HD21 ASN A   3       3.884  13.540   2.280  1.00  0.00           H  \n',
 'ATOM     13 HD22 ASN

In [22]:
with open('/net/storage/greedisgod/modules/write_pdb/write_pdb/residue_templates/asn.pdb_mod', 'w') as fh:
    fh.writelines(lines)